## Install dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install -e ../aicacia/extraction aicacia-document-exporter==0.1.4

## Extract from TEI files

In [ ]:
!cp ../aicacia/extraction/example/grobid/sample.grobid.tei.xml ../data/grobid

In [ ]:
from aicacia_extraction.grobid import TEIDocument

In [ ]:
tei_document = TEIDocument("../data/grobid/sample.grobid.tei.xml")

In [ ]:
tei_document.title

'Bi-criteria Algorithm for Scheduling Jobs on Cluster Platforms'

In [ ]:
tei_document.sections[0]

Section(title='INTRODUCTION 1.1 Cluster computing', text='The last few years have been characterized by huge technological changes in the area of parallel and distributed computing. Today, powerful machines are available at low price everywhere in the world. The main visible line of such changes is the large spreading of clusters which consist in a collection of tens or hundreds of standard almost identical processors connected together by a high speed interconnection network [6]. The next natural step is the extension to local sets of clusters or to geographically distant grids [10]. In the last issue of the Top500 ranking (from November 2003 [1]), 52 networks of workstations (NOW) of different kinds were listed and 123 entries are clusters sold either by IBM, HP or Dell. Looking at previous rankings we can see that this number (within the Top500) approximately doubled each year. This democratization of clusters calls for new practical administration tools. Even if more and more appli

In [ ]:
tei_document.figures[0]

Figure(title='Figure 1 :', label='1', description='Figure 1: Job submission in clusters.')